In [1]:
import pandas as pd

# Format the scores

In [18]:
scores = pd.read_csv('scores/auc_scores.csv')

In [19]:
scores = scores.rename(columns={'Unnamed: 0':'Method'})

In [25]:
scores

,index,Method,HC1125,HC1139,HC1302,HC1313,HC188,HC221,HC294,HC326,HC376,HC382,HC78
0,7,l1_log_reg SA,0.500000,0.500000,0.747738,0.500000,0.500000,0.782036,0.682895,0.674644,0.705679,0.500000,0.705562
1,1,l1_log_reg WB_only,0.614958,0.526826,0.836296,0.554659,0.635941,0.803022,0.732548,0.705413,0.676731,0.635612,0.597978
2,4,l1_log_reg WB_SA,0.614820,0.481254,0.834357,0.662933,0.629107,0.802445,0.733102,0.703324,0.688227,0.623633,0.611573
3,5,pretrained_lasso WB_SA,0.648892,0.603103,0.500000,0.598566,0.634923,0.500000,0.719806,0.500000,0.660388,0.644595,0.500000
4,3,glinternet WB_SA,0.638504,0.574661,0.834034,0.631123,0.659494,0.814379,0.746399,0.717759,0.736565,0.651963,0.702472


In [21]:
scores = scores.drop([0,2,6,8])

In [22]:
scores = scores.reset_index()

In [17]:
scores

,index,Method,HC1125,HC1139,HC1302,HC1313,HC188,HC221,HC294,HC326,HC376,HC382,HC78
0,7,l1_log_reg SA,0.500000,0.500000,0.747738,0.500000,0.500000,0.782036,0.682895,0.674644,0.705679,0.500000,0.705562
1,1,l1_log_reg WB_only,0.614958,0.526826,0.836296,0.554659,0.635941,0.803022,0.732548,0.705413,0.676731,0.635612,0.597978
2,4,l1_log_reg WB_SA,0.614820,0.481254,0.834357,0.662933,0.629107,0.802445,0.733102,0.703324,0.688227,0.623633,0.611573
3,5,pretrained_lasso WB_SA,NaN,NaN,0.500000,0.598566,0.634923,0.500000,0.719806,0.500000,0.660388,0.644595,0.500000
4,3,glinternet WB_SA,0.638504,0.574661,0.834034,0.631123,0.659494,0.814379,0.746399,0.717759,0.736565,0.651963,0.702472


In [24]:
scores.iloc[[1, 4]] = scores.iloc[[4, 1]].values

scores.iloc[[0, 1]] = scores.iloc[[1, 0]].values
#scores.iloc[[0, 1]] = scores.iloc[[1, 0]].values


In [26]:
scores = scores.drop(columns=['index'])

In [27]:
scores['Mean ROC-AUC'] = scores.iloc[:, 1:].mean(axis=1) * 100

In [28]:
scores['Increase'] = '+' + (round((scores['Mean ROC-AUC'] - scores['Mean ROC-AUC'][0]), 3)).astype(str)

In [29]:
scores[['Method','Dataset']] = scores.Method.str.split(expand=True) 

In [30]:
last_column = scores.columns[-1]
new_order   = [scores.columns[0], last_column] + list(scores.columns[1:-1])
scores      = scores[new_order]

In [31]:
scores

,Method,Dataset,HC1125,HC1139,HC1302,HC1313,HC188,HC221,HC294,HC326,HC376,HC382,HC78,Mean ROC-AUC,Increase
0,l1_log_reg,SA,0.500000,0.500000,0.747738,0.500000,0.500000,0.782036,0.682895,0.674644,0.705679,0.500000,0.705562,61.805028,+0.0
1,l1_log_reg,WB_only,0.614958,0.526826,0.836296,0.554659,0.635941,0.803022,0.732548,0.705413,0.676731,0.635612,0.597978,66.545314,+4.74
2,l1_log_reg,WB_SA,0.614820,0.481254,0.834357,0.662933,0.629107,0.802445,0.733102,0.703324,0.688227,0.623633,0.611573,67.134325,+5.329
3,pretrained_lasso,WB_SA,0.648892,0.603103,0.500000,0.598566,0.634923,0.500000,0.719806,0.500000,0.660388,0.644595,0.500000,59.184304,+-2.621
4,glinternet,WB_SA,0.638504,0.574661,0.834034,0.631123,0.659494,0.814379,0.746399,0.717759,0.736565,0.651963,0.702472,70.066843,+8.262


In [34]:
scores = scores.apply(pd.to_numeric, errors='ignore')

disease_code_columns = ['HC1125', 'HC1139', 'HC1302', 'HC1313', 
                        'HC188', 'HC221', 'HC294', 'HC326', 'HC376', 
                        'HC382', 'HC78','Mean ROC-AUC']

for col in disease_code_columns:
    scores[col] = scores[col].map(lambda x: '{:.3g}'.format(x))

In [35]:
def highlight_max(data, color='bold'):
    attr = f"font-weight: {color}"
    if data.ndim == 1:  # Single column
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # DataFrame
        is_max = data == data.max().reindex(data.columns, axis=1)
        is_max.loc['Method'] = False  # Exclude 'Method' row from highlighting
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

scores = scores.style.apply(highlight_max, subset=pd.IndexSlice[:, disease_code_columns])

In [36]:
scores

,Method,Dataset,HC1125,HC1139,HC1302,HC1313,HC188,HC221,HC294,HC326,HC376,HC382,HC78,Mean ROC-AUC,Increase
0,l1_log_reg,SA,0.5,0.5,0.748,0.5,0.5,0.782,0.683,0.675,0.706,0.5,0.706,61.8,+0.0
1,l1_log_reg,WB_only,0.615,0.527,0.836,0.555,0.636,0.803,0.733,0.705,0.677,0.636,0.598,66.5,+4.74
2,l1_log_reg,WB_SA,0.615,0.481,0.834,0.663,0.629,0.802,0.733,0.703,0.688,0.624,0.612,67.1,+5.329
3,pretrained_lasso,WB_SA,0.649,0.603,0.5,0.599,0.635,0.5,0.72,0.5,0.66,0.645,0.5,59.2,+-2.621
4,glinternet,WB_SA,0.639,0.575,0.834,0.631,0.659,0.814,0.746,0.718,0.737,0.652,0.702,70.1,+8.262
